In [3]:
import torch
import torch.nn as nn 
import torch.nn.functional as F

In [24]:
# A better approch will be to create entire list for all the layers
KERNEL = 3
STRIDE = 2
CHANNEL = [3,64, 64, 128, 128, 256, 256, 512, 512, 512, 512, 512, 512]
FC = [512*7*7, 4096, 4096 ]
POOL_POS = [2,4,6,9,12]  #after 2nd layer...

In [33]:
class VGG16(nn.Module):
    
    def __init__(self, num_of_classes = 1000, CHANNEL = CHANNEL , FC = FC, KERNEL = KERNEL, STRIDE = STRIDE, POOL_POS = POOL_POS ):
        super().__init__()
        
        self.layers = nn.ModuleList()
        self.fc = nn.ModuleList()
        self.flatten = nn.Flatten()
        self.classifier = nn.Linear(4096, num_of_classes)
        
        for i in range(1, len(CHANNEL)):
            # conv 2d layers
            self.layers.append(nn.Conv2d(in_channels=CHANNEL[i-1] , out_channels=CHANNEL[i], kernel_size= KERNEL, padding= 'same'))
            
            #activation layer
            self.layers.append(nn.ReLU(inplace = True))
            
            # Max pool
            if i in POOL_POS  :
                self.layers.append(nn.MaxPool2d(kernel_size= 2, stride = STRIDE))
         
        #Fully connected Layers              
        for i in range(len(FC)-1):
            self.fc.append(nn.Linear(FC[i], FC[i+1]))
            nn.ReLU()
        
    
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        
        x = self.flatten(x)
    
        for layer in self.fc:
            x = layer(x)
        
        #classifier call
        x = self.classifier(x)
        x = F.softmax(x)
        return x         

In [34]:
model = VGG16()
input = torch.randn(1, 3, 224, 224)